# GA DAT16
## Databases: SQLite (SQL) and MongoDB (noSQL)
### Justin Breucop, Francesco Mosconi

#### Adapted from @dowstreet's SF DAT5 Lab, Michael Keba DAT10 and @justalfred's NYC DS11 notebooks

This lab will cover two different types of databases:

- [SQL](https://en.wikipedia.org/wiki/SQL): table-oriented databases for relational or structured data
- [noSQL](https://en.wikipedia.org/wiki/Nosql): document-oriented databases for semi-structured or unstructured data

We will use SQLite as an example of an SQL database, and MongoDB as an example of a noSQL database.

### SQLite (SQL)

**[SQLite](https://sqlite.org/)** is a database software package built on the Structured Query Language [(SQL)](https://en.wikipedia.org/wiki/SQL).  It is similar to other SQL databases, such as [PostgreSQL](http://www.postgresql.org/), [MySQL](https://www.mysql.com/), Oracle, and Microsoft SQL Server, except that it is *file-based*, rather than *server-based*.  This makes it easy to setup and use for small projects, but less suitable for production environments.  Once you are familiar with sqlite, the same ideas, and similar syntax, can be applied to other SQL databases.

SQLite v3 is bundled with most python distributions (including our Anaconda distribution).  You might also find it useful to install [SQLite Manager](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/?src), a Firefox add-on for viewing SQLite database files via a simple GUI.

#### Interacting with SQLite
There are multiple ways of interacting with an SQLite database, including:

1. SQLite Command Line Utility
2. python `sqlite3` package
3. `pandas` SQL Interface
4. High-level ORMs (e.g. sqlalchemy, django ORM, etc.)

In this lab we will walk through methods 1-3, and then give pointers to a few *Object Relational Mappers (ORMs)* of method 4.  All of these methods provide some form of wrapper, or set of convenience functions, for interacting with SQLite.  Behind the scenes, the Structured Query Language (SQL) itself defines the interface to the database software.  This underlying SQL syntax will be visible to a greater or lesser degree depending upon the method that is chosen.

#### Common SQL Command Patterns
The SQL command set has a rich syntax with numerous options, but most of the commonly used commands follow a few simple patterns.  A basic familiarity of these patterns is helpful when working in SQL: 

    CREATE TABLE ...
    ALTER TABLE ... ADD COLUMN ...
    INSERT INTO ... VALUES ...
    UPDATE ... SET ... WHERE ...
    SELECT ... FROM ... WHERE ...
    SELECT ... FROM ... JOIN ... ON ...
    DELETE FROM ... WHERE ...

### 1. SQLite Command Line Utility

The first method we'll explore is connecting to SQLite via the built-in [command line utility](https://www.sqlite.org/sqlite.html).  

*Note: the commands in this section should be executed within your normal terminal shell, not in the python interpreter or an ipython notebook.  If you want to execute sql shell commands inside the notebook instead, you can install ipython-sql and use the sql magic syntax (but we're not covering that here...)*

To start a new session of the interpreter, simply open your terminal and type `sqlite3`, followed by the name of the database file.  If the file does not yet exist, sqlite will create it.

    $ sqlite3 test1.sqlite
    
    SQLite version 3.7.12 2012-04-03 19:43:07
    Enter ".help" for instructions
    Enter SQL statements terminated with a ";"
    sqlite> 

Notice that your terminal prompt changes to `sqlite>`, indicating that you are now entering commands into the sqlite command line utility.  Take a quick look at the help command:

    sqlite> .help
Display the current databases - you should see the new file `test1.db`:

    sqlite> .databases

#### Creating tables and adding columns
Create an table called `table1` with a single column `field1` containing an INTEGER PRIMARY KEY:

    sqlite> CREATE TABLE table1 (field1 INTEGER PRIMARY KEY);

Add a few more columns to `table1`: 

    sqlite> ALTER TABLE table1 ADD COLUMN field2 VARCHAR(16);
    sqlite> ALTER TABLE table1 ADD COLUMN field3 REAL;
    sqlite> ALTER TABLE table1 ADD COLUMN field4 TEXT;
    
Notice the different field types in the ALTER TABLE commands.  SQLite supports several different [field types](https://www.sqlite.org/datatype3.html), including INTEGERS, variable length VARCHAR character fields (with a max length), TEXT fields, and 'REALS', which are used to store floating point numbers.

Verify that the table was created:

    sqlite> .tables

You can check the `schema` of the table using `.schema`, which shows the commands that would be needed to create the database tables from scratch.  

    sqlite> .schema
    
Notice that in this case, our `table1` could have been created with a single command, rather than adding each column separately.


#### Adding data
Let's add some data:

    sqlite> INSERT INTO table1 VALUES (1, 'Henry James', 42, '75 Mission Street, San Francisco, CA');
    sqlite> INSERT INTO table1 VALUES (2, 'Carol James', 40, '75 Mission Street, San Francisco, CA');
    sqlite> INSERT INTO table1 VALUES (3, 'Jesse James', 12, '75 Mission Street, San Francisco, CA');

Notice that the first column has unique values - this is a requirement for the PRIMARY KEY column.  If we try to add a record using an existing PK value we'll get an error:

    sqlite> INSERT INTO table1 VALUES (3, 'Julie James', 10, '75 Mission Street, San Francisco, CA');
    Error: PRIMARY KEY must be unique

Fortunately, SQLite has some built in functionality to auto-increment the PK value - just set the value of the PK field to NULL when doing the INSERT and it will automatically be set to a valid value.

    sqlite> INSERT INTO table1 VALUES (NULL, 'Julie James', 10, '75 Mission Street, San Francisco, CA');

Now that we have some data, take a look at the database using the **SQLite Manager** Firefox plugin.

1. Firefox -> Tools -> SQLite Manager
2. Select Connect Database
3. Highlight the table and then click the Browse and Search tab

Notice that the value in `field1` for the Julie James record has been automatically set to 4. 

#### Updating records
Suppose we need to update an existing record with new data - e.g. maybe Julie James is only 9.  For this we use the UPDATE command:

    sqlite> UPDATE table1 SET field3=9 WHERE field1=4;

#### Basic Queries
As the second term in Structured Query Language implies, a real strength of SQL is the ability to perform complex queries, returning a subset of records that match particular criteria.  Let's try a couple:

    sqlite> SELECT * FROM table1 WHERE field1 = 2;
    2|Carol James|40.0|75 Mission Street, San Francisco, CA

    sqlite> SELECT * FROM table1 WHERE field3 > 35;
    1|Henry James|42.0|75 Mission Street, San Francisco, CA
    2|Carol James|40.0|75 Mission Street, San Francisco, CA

    sqlite> SELECT * FROM table1 WHERE field2 like '%Henry%';
    1|Henry James|42.0|75 Mission Street

    sqlite> SELECT * FROM table1 WHERE field2 like '%James%' and field3 < 15;
    3|Jesse James|12.0|75 Mission Street, San Francisco, CA
    4|Julie James|9.0|75 Mission Street, San Francisco, CA
    

#### Removing Records
To remove records use the DELETE command:

    sqlite> DELETE FROM table1 WHERE field2 like '%Jesse%';
    
Use SQLite-Manager to verify that the Jesse James record has been removed.  To exit the sqlite interpreter type `.exit`.

    sqlite>  .exit

### 2. python `sqlite3` package

The command line utility can be useful for basic SQL tasks, but since we're using python for the rest of code it will often be easier to access sqlite directly from within python.  We can use the python [`sqlite3`](https://docs.python.org/2.7/library/sqlite3.html) package for just this purpose.

*Note: from this point forward we'll be running python commands within the notebook as usual.*

In [ ]:
#usual imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.plotting import figure,show,output_notebook
output_notebook()
# our new package
import sqlite3

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [ ]:
sqlite_db = 'housing.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

The syntax to create a table is similar to before, only now we use the `execute` method of the cursor object `c` that we just created:

In [ ]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

In [ ]:
# Save (commit) the changes
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitutionusing the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [ ]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

In [ ]:
# Remember to commit the changes
conn.commit()

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

In [ ]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Once again, use SQLite Manager to verify the database contents. 

#### Adding data from a csv file
Next let's load our housing.csv data into an array, and then `INSERT` those records into the database.  In this example we'll use the numpy `genfromtxt` function to read the file and parse the contents. 

In [ ]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = (genfromtxt('housing-data.csv', dtype='i8', 
                    delimiter=',', skip_header=1)).tolist()

# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

# confirm that the data looks the way we want it to
data

In [ ]:
# loop through data, running an INSERT on each record (i.e. sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

In [ ]:
conn.commit()

A reason for this example - remember that all elements in a numpy array must be the same data type, so if we want to 'add a None' to each row, we need to work around this.  Lists can contain mixed types, so that is one approach.

Still, in this case the value we're adding is the same for all records, so we could have simply used a 'None' in the INSERT statement directly:

In [ ]:
data = genfromtxt('houses.csv', dtype='i8', 
                   delimiter=',', skip_header=1)

In [ ]:
data

In [ ]:
# for d in data:
#    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', [None, d[0], d[1], d[2], d[3]])

# conn.commit()

Unfortunately we now have duplicate records in our database.  Here's a DELETE string to remove them:

In [ ]:
# c.execute('DELETE FROM houses WHERE pk not in \
#         (SELECT min(pk) FROM houses GROUP BY sqft, bdrms, age, price)') 

conn.commit()

#### More Queries
Let's try some queries with this slightly larger dataset, this time using the python syntax.

In [ ]:
# similar syntax as before
# results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# here results is a cursor object - use fetchall() to extract a list
# results.fetchall()

In [ ]:
# ...

Try some of your own queries ...

#### Multi-table Datasets - ENRON Archive

So far our sample databases have had only a single table, but SQL is really oriented around multi-table databases.  Let's load a database with a more complex schema - the email archive of the ENRON corporation.

In [ ]:
conn = sqlite3.connect('enron.db') 
c = conn.cursor()

Take a look at the database in SQLite Manager - you should see 3 tables.  To view a table list in python we can query the master table:

In [ ]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

In [ ]:
fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='EmployeeBase';").fetchall()
print fields

In [ ]:
# make it a little easier to look at
fields =(fields[0][0].split('\n'))[1:]  # pull out the string and split on \n into list
print fields

The cursor also has a description attribute with this data, that may be easier to work with:

In [ ]:
c.description

In [ ]:
[f[0] for f in c.description] 

Look at the first 5 records of each table:

In [ ]:
# print first 5 rows of EmployeeBase table
results = c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
for row in results:
   print row

In [ ]:
# print first 5 rows of MessageBase table
results = c.execute("SELECT * FROM MessageBase LIMIT 5;").fetchall()
for row in results:
   print row

In [ ]:
# print first 10 rows of RecipientBase table
results = c.execute("SELECT * FROM RecipientBase LIMIT 10;").fetchall()
for row in results:
   print row
    
# mid, rno, to_eid    

The first field is message id, the second is recipient number, and the third is the id of the recipient.  Let's do a quick count of the number of recipients per message.

In [ ]:
results = c.execute("SELECT max(rno) FROM RecipientBase GROUP BY mid;").fetchall()
print results

In [ ]:
# use a python 2.7 convenience function: collections
import collections

counts = collections.Counter(results)

print counts

In [ ]:
print(counts.most_common())

In [ ]:
x = [i[0][0] for i in counts.most_common()]
y = [i[1] for i in counts.most_common()]
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

Rescale to investigate the tail of the curve

In [ ]:
x = [i[0][0] for i in counts.most_common()[5:]] # chop off the first 5
y = [i[1] for i in counts.most_common()[5:]]  # chop off the first 5
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

Let's perform a similar count of which employees received the most messages: 

In [ ]:
results = c.execute("SELECT to_eid, count(to_eid) FROM RecipientBase GROUP BY to_eid;").fetchall()
results.sort(key=lambda x: x[1], reverse=True)
print results

Top recipients?

In [ ]:
# print c.execute("SELECT * FROM EmployeeBase WHERE eid = 131;").fetchall()

In [ ]:
# print c.execute("SELECT * FROM EmployeeBase WHERE eid = 122;").fetchall()

In [ ]:
# print c.execute("SELECT * FROM EmployeeBase WHERE eid = 138;").fetchall()

Bottom recipients?

In [ ]:
# print c.execute("SELECT * FROM EmployeeBase WHERE eid = 52;").fetchall()

In [ ]:
# print c.execute("SELECT * FROM EmployeeBase WHERE eid = 139;").fetchall()

What does this distribution look like?

In [ ]:
x = [i[0]for i in results]
y = [i[1] for i in results]
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Popular Employees",tools='',x_axis_label='# received',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

#### JOINS

It looks like RecipentBase is a JOIN table (i.e. contains FOREIGN KEYS) used to link records in EmployeeBase and MessageBase (which is in fact the case).  We can do a SELECT that first JOINS the data of two different tables.  Notice that the returned records contain fields from two different tables (but not all the fields).

In [ ]:
# results = c.execute("SELECT name, department, title, seniority, subject, filename \
#           FROM EmployeeBase JOIN MessageBase ON eid = from_eid LIMIT 5;").fetchall()
# for row in results:
#    print row

Try some of your own queries

### 3. `pandas` SQL Interface
`pandas` has some convenience functions that simplify the interface to SQLite.  Under the hood, `pandas` is still using the `sqlite3`, but these functions take care of some of the data wrangling for you.  

In [ ]:
import pandas as pd

Let's start with a 'manual' approach to better understand the transformation:

In [ ]:
results = c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
for row in results:
    print row

You can pass this list of tuples to the DataFrame constructor, but you also need column names, contained in the cursor's `description` attribute.  

ref: [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do), p.175

In [ ]:
# c.description

With these two pieces of info we can create a dataframe with the results:

In [ ]:
pd.DataFrame(results, columns=zip(*c.description)[0])

The `read_frame` function in the `pandas.io.sql` module simplifies the process a bit more:

In [ ]:
import pandas.io.sql as sql

In [ ]:
# pass in the query string and the connection as arguments
sql.read_sql('SELECT * FROM EmployeeBase LIMIT 5', conn)

In [ ]:
sql.read_sql('SELECT * FROM MessageBase LIMIT 5', conn)

In [ ]:
sql.read_sql('SELECT * FROM RecipientBase LIMIT 5', conn)

#### More JOINS
You can link JOINs together to build very complex logic that spans tables.  What if we want to build a DataFrame that has message id, sender ID, sender name, receiver ID, receiver name, and subject?

In [ ]:
sql.read_sql('SELECT * FROM RecipientBase JOIN EmployeeBase ON to_eid=eid LIMIT 5', conn) 

In [ ]:
sql.read_sql('SELECT * FROM RecipientBase JOIN MessageBase ON RecipientBase.mid=MessageBase.mid LIMIT 5', conn) 

Fetching values from the same table multiple times

In [ ]:
# JOINS ACROSS MULTIPLE TABLES - CONFUSING!
emails = sql.read_frame('SELECT \
                        rb1.mid as msg_id, \
                        mb1.from_eid as from_id, \
                        eb2.name as from_name, \
                        rb1.to_eid as to_id, \
                        eb1.name as to_name, \
                        mb1.subject \
                FROM RecipientBase as rb1 \
                INNER JOIN EmployeeBase as eb1 ON rb1.to_eid=eb1.eid \
                INNER JOIN EmployeeBase as eb2 ON mb1.from_eid = eb2.eid \
                INNER JOIN MessageBase as mb1 ON rb1.mid = mb1.mid' \
                , conn)

In [ ]:
emails

Which employees sent the most 'mass' emails?

In [ ]:
sql.read_frame('SELECT name, title, MAX(rno) AS recipient_count \
                FROM EmployeeBase \
                JOIN MessageBase ON eid = from_eid \
                JOIN RecipientBase USING(mid) \
                GROUP BY name, title ORDER BY recipient_count DESC LIMIT 5', conn)

#### Writing from a DataFrame back to SQLite
Let's pretend the Trading department has been renamed the Forestry department.  How can we update all the records in the EmployeeBase table?

In [ ]:
people = sql.read_sql('SELECT * FROM EmployeeBase', conn)
people.head()

In [ ]:
# ...
# ...

In [ ]:
sql.to_sql(people, 'EmployeeBase', conn, if_exists='replace')    # if_exists = <'fail', 'replace', 'append'>

In [ ]:
sql.read_sql('SELECT * FROM EmployeeBase', conn)

### 4. High-level ORMs (e.g. sqlalchemy, django ORM, etc.)

If you do any significant SQL programming in python, you may find it convenient to use a library or framework that provides a higher-level abstraction of your database.  Such libraries often include an Object Relational Mapper (ORM), which allows you to work with data in terms of python objects or classes, transparently handling database operations in the background.  A few examples are included here for reference:

http://www.sqlalchemy.org/  
http://www.djangoproject.com/  
http://python-orm.org
http://curdpy.readthedocs.org/en/latest/  
https://wiki.python.org/moin/HigherLevelDatabaseProgramming  

Some of these libraries provide a single abstract interface that hides the differences between various SQL implementations - e.g. SQLite, PostgreSQL, MySQL, etc.  This can make it easier to transfer code between environments, for example, if you are using SQLite during development but another server for production.

#### SQL via REST API / JSON

Another very common approach for accessing an SQL database is to run a webserver that (i) connects to the database on the backend and (ii) presents a REST / JSON API over HTTP.  Queries are then submitted as specially formed URL strings, and results are returned as JSON objects (which are easily converted to python dictionaries).

#### One more trick - serializing JSON into SQL TEXT

In [ ]:
# data fetching, parsing, and storage
import urllib2
import json

url = 'https://www.govtrack.us/api/v2/vote/1'

In [ ]:
def fetch_json(url):
    """ Fetch a json file / object from a url and convert to a python dict

    Args:
        url:  url of the json object

    Returns:
        data:  python dict containing fetched data

    """
    req = urllib2.Request(url, None, {'user-agent': 'syncstream/vimeo'})
    opener = urllib2.build_opener()
    f = opener.open(req)
    # data = simplejson.load(f)
    data = json.load(f)
    return data


In [ ]:
data = fetch_json(url)
data

In [ ]:
conn = sqlite3.connect('json_test.db') 
c = conn.cursor()

# create a simple table
c.execute("CREATE TABLE records (pk INTEGER PRIMARY KEY, json TEXT)")

In [ ]:
conn.commit()

In [ ]:
json_rec = json.dumps(data)
record = (None, json_rec)

In [ ]:
c.execute('INSERT INTO records VALUES (?, ?)', record)

In [ ]:
conn.commit()

In [ ]:
out = c.execute('SELECT * FROM records').fetchall()

In [ ]:
data_out = json.loads(out[0][1])

In [ ]:
data_out

## MongoDB (noSQL)

Mongo can be handy for quick personal projects. It's quick to set up and you don't have to think about designing schemas. Just don't expect it to scale well to a hundred servers. If you want to get a flavor for connecting to Mongo from Python, complete this section.

If you're on a Mac, the following should work....

```
brew install mongodb # if this doesn't work try sudo chown -R $USER /usr/local
pip install pymongo  # if this doesn't work, try sudo pip install pymongo
mkdir ~/temp_mongo
mongod --dbpath ~/temp_mongo
```

For Linux users, find your distro here: http://www.mongodb.org/downloads#packages
The latter three steps should be the same.

For Windows users, you can download binaries here: http://www.mongodb.org/downloads

We'll use the Rotten Tomatoes dataset.

In [ ]:
# load data from a csv file into a Pandas DataFrame.

df = pd.read_csv('./data/rt_critics.csv', 
                 parse_dates=[5],
                 encoding='utf-8')

In [ ]:
from pymongo import MongoClient

connection = MongoClient('localhost', 27017)
db = connection['GA']
collection = db['tomato']

In [ ]:
collection.count()

In [ ]:
# store records in mongo
for _, rec in df.iterrows():
    collection.insert(rec.to_dict())

In [ ]:
# fetch the records back out. What do they look like?
print collection.find({}).count(), 'total records'

df = pd.DataFrame.from_records(collection.find({}), index='_id')
df.head()

**Write some Mongo queries**

Note: pymongo is slightly different from the javascript examples you went through in the tutorials. Most notably, the keys in the dictionary need to be encapsulated in quotes in python. You will probably want to refer the [pymongo reference](http://api.mongodb.org/python/current/) for this section. If you're not used to learning an API from reading the docs, this section is probably not worth your effort. Mongo isn't worth that much pain.

In [ ]:
# How many reviews are by Roger Ebert?

In [ ]:
# How many fresh reviews are by Roger Ebert?

In [ ]:
# Find one rotten review by Roger Ebert

In [ ]:
# Find all rotten reviews by Roger Ebert. Store in a DataFrame as in the example and print the head.

In [ ]:
# In which publications has Roger Ebert made fresh reviews?
# This one will probably require munging in python instead of pure Mongo.

In [ ]:
# How many reviews are from before the year 2000?
from datetime import datetime

### References & Further Reading:

[sqlite3 home](http://www.sqlite.org)  
[SQLite - Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
[SQLAlchemy home](http://www.sqlalchemy.org)  
[MongoDB home](http://www.mongodb.org/)
[CAP Theorem](http://en.wikipedia.org/wiki/CAP_theorem)
[Response to CAP Theorem](http://www.infoq.com/articles/cap-twelve-years-later-how-the-rules-have-changed)